<a href="https://colab.research.google.com/github/omarbadr1/AI-agent-container-tracker/blob/main/Container_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [8]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [9]:
from crewai import Agent, Task, Crew

In [10]:
import os
from getpass import getpass

openai_api_key = getpass("Enter your OpenAI API key: ")
serper_api_key = getpass("Enter your Serper API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

Enter your OpenAI API key: ··········
Enter your Serper API key: ··········


In [20]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, SeleniumScrapingTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
selenium_tool = SeleniumScrapingTool()


In [13]:
Webscraper = Agent(
    role="Web Scraper",
    goal="Search for and retrieve up-to-date ETA information for containers in transit.",
    backstory=(
        "You are a senior logistics coordinator at an FMCG company. "
        "Your primary responsibility is to monitor and gather updates on the estimated time of arrival (ETA) "
        "for shipping containers currently in transit. "
    ),
    allow_delegation=False,
    verbose=True
)


In [14]:
writer = Agent(
    role="Logistics Communication Specialist",
    goal="Summarise container ETA updates from the Webscraper and communicate them clearly and concisely to relevant stakeholders.",
    backstory=(
        "You are a senior logistics communication specialist responsible for ensuring all stakeholders are informed "
        "about the latest estimated time of arrival (ETA) for containers in transit. You work closely with the Webscraper, "
        "who gathers real-time ETA updates, primarily focusing on shipments from Maersk. "
        "Your job is to translate these updates into concise, professional messages that are easy to understand. "
        "Your communication should be accurate, actionable, and tailored for time-sensitive logistics operations."
    ),
    allow_delegation=False,
    verbose=True
)


In [38]:
Scrap = Task(
    description=(
        "Retrieve the ETA for container MRSU6874308 from the Maersk tracking page.\n"
        "- Visit: https://www.maersk.com/tracking/MRSU6874308\n"
        "- Accept all cookies if prompted.\n"
        "- Look for the ETA inside a <slot> tag with class='sublabel', name='sublabel', and part='text-and-icon-sublabel'.\n"
        "- Extract the text inside that slot element, e.g., '19 May 2025 15:00'.\n"
        "- Also try to determine if the container is 'In Transit' or 'In Port'.\n"
    ),
    expected_output="Container number: MRSU6874308 | Status: [In Port / In Transit] | ETA: [Date & Time]",
    tools=[selenium_tool],
    human_input=False,
    agent=Webscraper,
)


In [39]:
write = Task(
    description=(
        "1. Use the container info to provide a clear update on the container status and ETA "
        "2. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written message ",
    agent=writer,
)

In [40]:
crew = Crew(
    agents=[Webscraper, writer],
    tasks=[Scrap, write],
    verbose=2
)

In [41]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Web Scraper
 [INFO]: == Starting Task: Retrieve the ETA for container MRSU6874308 from the Maersk tracking page.
- Visit: https://www.maersk.com/tracking/MRSU6874308
- Accept all cookies if prompted.
- Look for the ETA inside a <slot> tag with class='sublabel', name='sublabel', and part='text-and-icon-sublabel'.
- Extract the text inside that slot element, e.g., '19 May 2025 15:00'.
- Also try to determine if the container is 'In Transit' or 'In Port'.



> Entering new CrewAgentExecutor chain...
I need to read the content of the Maersk tracking page for container MRSU6874308 to find the ETA and status of the container.

Action: Read a website content
Action Input: 
{
  "website_url": "https://www.maersk.com/tracking/MRSU6874308",
  "css_element": "slot.sublabel[name='sublabel'][part='text-and-icon-sublabel']"
}
 



Thought: I need to read the content of the Maersk tracking page for container MRSU6874308 to find the ETA and status of the container.

Action: